In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *


In [ ]:
#| code-summary: import all the packages needed for the project

from ids_finder.utils import *
from ids_finder.core import *
from fastcore.utils import *
from fastcore.test import *

import polars as pl
import xarray as xr
try:
    import modin.pandas as pd
    import modin.pandas as mpd
except ImportError:
    import pandas as pd

import pandas
import numpy as np
from xarray_einstats import linalg

from datetime import timedelta

from loguru import logger


import pytplot
from pytplot import timebar
from pytplot import get_data, store_data, tplot, split_vec, join_vec, tplot_options, options, tlimit, highlight, degap

import pdpipe as pdp


from collections.abc import Callable
from pandas import (
    DataFrame,
    Timestamp,
)
from xarray.core.dataarray import DataArray

## Background

Spacecraft-Solar equatorial

### Coordinate System of Data


1. **SE (Solar Equatorial)**
    - Code: `se`
    - Resampling options: 
        - Number of seconds (1 or 60): `se_rN[N]s`
        - Resampled 1 hour: `se_r1h`

2. **PC (Planetocentric)**
    - Code: `pc`
    - Resampling options: 
        - Number of seconds (1 or 60): `pc_rN[N]s`
        
3. **SS (Sun-State)**
    - Code: `ss`
    - Resampling options: 
        - Number of seconds (1 or 60): `ss_rN[N]s`
        
4. **PL (Payload)**
    - Code: `pl`
    - Resampling options: 
        - Number of seconds (1 or 60): `pl_rN[N]s`


```txt
------------------------------------------------------------------------------
Juno Mission Phases                                                           
------------------------------------------------------------------------------
Start       Mission                                                           
Date        Phase                                                             
==============================================================================
2011-08-05  Launch                                                            
2011-08-08  Inner Cruise 1                                                    
2011-10-10  Inner Cruise 2                                                    
2013-05-28  Inner Cruise 3                                                    
2013-11-05  Quiet Cruise                                                      
2016-01-05  Jupiter Approach                                                  
2016-06-30  Jupiter Orbital Insertion                                         
2016-07-05  Capture Orbit                                                     
2016-10-19  Period Reduction Maneuver                                         
2016-10-20  Orbits 1-2                                                        
2016-11-09  Science Orbits                                                    
2017-10-11  Deorbit
```

```txt
File Naming Convention                                                        
==============================================================================
Convention:                                                                   
   fgm_jno_LL_CCYYDDDxx_vVV.ext                                               
Where:                                                                        
   fgm - Fluxgate Magnetometer three character instrument abbreviation        
   jno - Juno                                                                 
    LL - CODMAC Data level, for example, l3 for level 3                       
    CC - The century portion of a date, 20                                    
    YY - The year of century portion of a date, 00-99                         
   DDD - The day of year, 001-366                                             
    xx - Coordinate system of data (se = Solar equatorial, ser = Solar        
         equatorial resampled, pc = Planetocentric, ss = Sun-State,           
         pl = Payload)                                                        
     v - separator to denote Version number                                   
    VV - version                                                              
   ext - file extension (sts = Standard Time Series (ASCII) file, lbl = Label 
         file)                                                                
Example:                                                                      
   fgm_jno_l3_2014055se_v00.sts    
```

## Dataset Overview

In [ ]:
pds_dir = "https://pds-ppi.igpp.ucla.edu/data"

possible_coords = ["se", "ser", "pc", "ss", "pl"]
possible_exts = ["sts", "lbl"]
possible_data_rates = ["1s", "1min", "1h"]

juno_ss_config = {
    "DATA_SET_ID": "JNO-SS-3-FGM-CAL-V1.0",
    "FILE_SPECIFICATION_NAME": "INDEX/INDEX.LBL",
}

juno_j_config = {
    "DATA_SET_ID": "JNO-J-3-FGM-CAL-V1.0",
    "FILE_SPECIFICATION_NAME": "INDEX/INDEX.LBL",
}

In [ ]:
#| hide
def download_and_read_lbl_file(config, index_table=False):
    """Download and read file for each config.

    Returns:
        DataFrame: The data read from the file.
    """
    # BUG: index file is not formatted properly according to `lbl` file, so can not be used with `pdr` for.
    # ValueError: time data "282T00:00:31.130,2019" doesn't match format "%Y-%jT%H:%M:%S.%f", at position 3553. You might want to try:
    # - passing `format` if your strings have a consistent format;
    # - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    # - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

    local_dir = os.path.join(os.environ["HOME"], "juno", config["DATA_SET_ID"])
    base_url = f"{pds_dir}/{config['DATA_SET_ID']}"

    lbl_fn = config["FILE_SPECIFICATION_NAME"]

    if not index_table:
        parquet_fn = lbl_fn.replace("lbl", "parquet")
        parquet_fp = os.path.join(local_dir, parquet_fn)
        if os.path.exists(parquet_fp):
            return pandas.read_parquet(os.path.join(local_dir, parquet_fn))

    lbl_file_url = f"{base_url}/{lbl_fn}"
    lbl_fp = download_file(lbl_file_url, local_dir, lbl_fn)
    logger.debug(f"Reading {lbl_fp}")

    if index_table:
        tab_fn = lbl_fn.replace("LBL", "TAB")
        tab_fp = download_file(f"{base_url}/{tab_fn}", local_dir, tab_fn)
        tab_index = pandas.read_csv(tab_fp, delimiter=",", quotechar='"')
        tab_index.columns = tab_index.columns.str.replace(" ", "")
        return tab_index
    else:
        sts_fn = lbl_fn.replace("lbl", "sts")
        download_file(f"{base_url}/{sts_fn}", local_dir, sts_fn)
        return pdr.read(lbl_fp).TABLE

In [ ]:
#| hide
jno_ss_index = download_and_read_lbl_file(juno_ss_config, index_table=True)
jno_j_index = download_and_read_lbl_file(juno_j_config, index_table=True)

_index_time_format = "%Y-%jT%H:%M:%S.%f"

jno_pipeline = pdp.PdPipeline(
    [
        pdp.ColDrop(["PRODUCT_ID", "CR_DATE", "PRODUCT_LABEL_MD5CHECKSUM"]),
        pdp.ApplyByCols("SID", str.rstrip),
        pdp.ApplyByCols("FILE_SPECIFICATION_NAME", str.rstrip),
        pdp.ColByFrameFunc(
            "START_TIME",
            lambda df: pandas.to_datetime(df["START_TIME"], format=_index_time_format),
        ),
        pdp.ColByFrameFunc(
            "STOP_TIME",
            lambda df: pandas.to_datetime(df["STOP_TIME"], format=_index_time_format),
        ),
        # pdp.ApplyByCols(['START_TIME', 'STOP_TIME'], pandas.to_datetime, format=_index_time_format), # NOTE: This is slow
    ]
)

jno_ss_index = jno_pipeline(jno_ss_index)
jno_j_index = jno_pipeline(jno_j_index)

index_df = pandas.concat(
    [jno_ss_index, jno_j_index], ignore_index=True
)

2023-09-23 10:04:08.643 | DEBUG    | __main__:download_and_read_lbl_file:27 - Reading /Users/zijin/juno/JNO-SS-3-FGM-CAL-V1.0/INDEX/INDEX.LBL
2023-09-23 10:04:08.665 | DEBUG    | __main__:download_and_read_lbl_file:27 - Reading /Users/zijin/juno/JNO-J-3-FGM-CAL-V1.0/INDEX/INDEX.LBL


### Check the data

In [ ]:
#| echo: false
starting_date = jno_ss_index['START_TIME'].min().date()
ending_date = jno_ss_index['STOP_TIME'].max().date()

print(f"Starting date: {starting_date}")
print(f"Ending date: {ending_date}")

Starting date: 2011-08-25
Ending date: 2016-06-29


In [ ]:
#| echo: false
available_dates = pandas.concat([jno_ss_index['START_TIME'].dt.date, jno_ss_index['STOP_TIME'].dt.date]).unique()
full_year_range = pandas.date_range(start=starting_date, end=ending_date)

missing_dates = full_year_range[~full_year_range.isin(available_dates)]

if len(missing_dates) == 0:
    print(f"No days are missing.")
else:
    print(f"The following days are missing")
    print(coll_repr(missing_dates.map(lambda x: x.strftime("%Y-%m-%d"))))

The following days are missing
(#251) ['2012-04-20','2012-04-21','2012-04-22','2012-04-23','2012-04-24','2012-05-15','2012-06-15','2012-07-04','2012-07-05','2012-07-06'...]


### Download all the files

In [ ]:
# wget -r --no-parent --no-clobber 'https://pds-ppi.igpp.ucla.edu/data/JNO-SS-3-FGM-CAL-V1.0/DATA/CRUISE/SE/1SEC/'
# aria2c -x 16 -s 16 'https://pds-ppi.igpp.ucla.edu/ditdos/download?id=pds://PPI/JNO-SS-3-FGM-CAL-V1.0/DATA/CRUISE/SE/1SEC'

1 day of data resampled by 1 sec is about 12 MB.

So 1 year of data is about 4 GB, and 6 years of JUNO Cruise data is about 24 GB.

Downloading rate is about 250 KB/s, so it will take about 3 days to download all the data.

In [ ]:
num_of_files = 6*365
jno_file_size = 12e3
thm_file_size = 40e3
files_size = jno_file_size + thm_file_size
downloading_rate = 250
processing_rate = 1/60

time_to_download = num_of_files * files_size / downloading_rate / 60 / 60
space_required = num_of_files * files_size / 1e6
time_to_process = num_of_files / processing_rate / 60 / 60

print(f"Time to download: {time_to_download:.2f} hours")
print(f"Disk space required: {space_required:.2f} GB")
print(f"Time to process: {time_to_process:.2f} hours")

Time to download: 126.53 hours
Disk space required: 113.88 GB
Time to process: 36.50 hours


### Convert all files from `lbl` to `parquet` for faster processing

In [ ]:
import pdr

In [ ]:
#| code-summary: Convert data from `lbl` to `parquet` format
def lbl2parquet(src: Path, dest: Path) -> None:
    df = pdr.read(src).TABLE
    df.to_parquet(dest)


def convert_file(
    file_path: Path, target_format: str, conversion_func: Callable, check_exist=True
) -> None:
    target_suffix = (
        target_format if target_format.startswith(".") else f".{target_format}"
    )
    target_file = file_path.with_suffix(target_suffix)

    if check_exist and target_file.exists():
        return True

    try:
        conversion_func(file_path, target_file)
    except Exception as e:
        logger.error(f"Error converting {file_path} to {target_file}: {e}")
        return False

@startthread
def convert_files():
    format_from = "lbl"
    format_to = "parquet"
    local_dir = Path.home() / "data/juno"
    pattern = f"**/*.{format_from}"
    convert_func = lbl2parquet
    for file in local_dir.glob(pattern):
        convert_file(file, format_to, convert_func)
    logger.info("Done converting files")

In [ ]:
# delete all files with extension
# find . -type f -name '*.parquet' -delete
# find . -type f -name '*.orc' -delete
# find . -type f -name '*.lbl' -delete

In [ ]:
def _batch_pre_process(year, force=False):
    trange = [f"{year}-01-01", f"{year+1}-01-01T01"]  # having some overlap
    dir_path = Path.home() /  "data/juno/JNO-SS-3-FGM-CAL-V1.0/"
    pattern = "**/*.parquet"
    data = dir_path / pattern
    
    output = Path(f"../data/jno_{year}.parquet")
    output.parent.mkdir(parents=True, exist_ok=True)
    if os.path.exists(output) and not force:
        logger.info(f"File {output} exists. Skipping")
        return output
    logger.info(f"Preprocessing data for year {year}")
    
    lazy_df = pl.scan_parquet(data)
    temp_df = (
        lazy_df.filter(
            pl.col("time").is_between(pd.Timestamp(trange[0]), pd.Timestamp(trange[1])),
        )
        .sort(
            "time"
        )  # needed for `compute_index_std` to work properly as `group_by_dynamic` requires the data to be sorted
        .filter(
            pl.col(
                "time"
            ).is_first_distinct()  # remove duplicate time values for xarray to select data properly, though significantly slows down the computation
        )
        .rename({"BX SE": "BX", "BY SE": "BY", "BZ SE": "BZ"})
    )
    temp_df.collect().write_parquet(output)
    return output

@startthread
def batch_pre_process():
    starting_year = starting_date.year
    ending_year = ending_date.year

    for year in range(starting_year, ending_year+1):
        _batch_pre_process(year)

2023-09-23 10:04:09.053 | INFO     | __main__:convert_files:33 - Done converting files
2023-09-23 10:04:09.072 | INFO     | __main__:_batch_pre_process:10 - File ../data/jno_2011.parquet exists. Skipping
2023-09-23 10:04:09.073 | INFO     | __main__:_batch_pre_process:10 - File ../data/jno_2012.parquet exists. Skipping
2023-09-23 10:04:09.074 | INFO     | __main__:_batch_pre_process:10 - File ../data/jno_2013.parquet exists. Skipping
2023-09-23 10:04:09.074 | INFO     | __main__:_batch_pre_process:10 - File ../data/jno_2014.parquet exists. Skipping
2023-09-23 10:04:09.075 | INFO     | __main__:_batch_pre_process:10 - File ../data/jno_2015.parquet exists. Skipping


2023-09-23 10:04:09.075 | INFO     | __main__:_batch_pre_process:10 - File ../data/jno_2016.parquet exists. Skipping


## Processing the whole data

In [ ]:
show_doc(process_candidates)

---

[source](https://github.com/Beforerr/planet/blob/master/ids_finder/core.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### process_candidates

>      process_candidates
>                          (candidates:modin.pandas.dataframe.DataFrame|pandas.c
>                          ore.frame.DataFrame,
>                          sat_fgm:xarray.core.dataarray.DataArray,
>                          sat_state:xarray.core.dataarray.DataArray,
>                          data_resolution:datetime.timedelta)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| candidates | modin.pandas.dataframe.DataFrame \| pandas.core.frame.DataFrame | potential candidates DataFrame |
| sat_fgm | DataArray | satellite FGM data |
| sat_state | DataArray | satellite state data |
| data_resolution | timedelta | time resolution of the data |

In [ ]:
from tqdm import tqdm

In [ ]:
sat = 'jno'
coord = 'se'
tau = timedelta(seconds=60)
data_resolution = timedelta(seconds=1)

@startthread
def batch_process():
    for year in tqdm(range(starting_date.year, ending_date.year+1)):
        files = f'../data/{sat}_{year}.parquet'
        output = f'../data/{sat}_candidates_{year}_tau_{tau.seconds}.parquet'
        
        if os.path.exists(output):
            logger.info(f"Skipping {year} as the output file already exists.")
            continue

        data = pl.scan_parquet(files).set_sorted('time').collect()
        sat_fgm = df2ts(data, ["BX", "BY", "BZ"], attrs={"coordinate_system": coord, "units": "nT"})
        sat_state = df2ts(data, ["X", "Y", "Z"], attrs={"coordinate_system": coord, "units": "km"})

        indices = compute_indices(data, tau)
        # filter condition
        sparse_num = tau / data_resolution // 3
        filter_condition = get_ID_filter_condition(sparse_num = sparse_num)

        candidates_pl = indices.filter(filter_condition).with_columns(pl_format_time(tau))
        candidates = convert_to_dataframe(candidates_pl)
        
        ids = process_candidates(candidates, sat_fgm, sat_state, data_resolution)
        
        if isinstance(ids, mpd.DataFrame):
            ids._to_pandas().to_parquet(output)

  0%|          | 0/6 [00:00<?, ?it/s]2023-09-23 10:16:25.411 | INFO     | __main__:batch_process:13 - Skipping 2011 as the output file already exists.


2023-09-23 10:16:25.412 | INFO     | __main__:batch_process:13 - Skipping 2012 as the output file already exists.
2023-09-23 10:16:25.413 | INFO     | __main__:batch_process:13 - Skipping 2013 as the output file already exists.
2023-09-23 10:16:25.414 | INFO     | __main__:batch_process:13 - Skipping 2014 as the output file already exists.
2023-09-23 10:16:25.414 | INFO     | __main__:batch_process:13 - Skipping 2015 as the output file already exists.
2023-09-23 10:16:25.415 | INFO     | __main__:batch_process:13 - Skipping 2016 as the output file already exists.
100%|██████████| 6/6 [00:00<00:00, 1551.63it/s]

#### THEMIS

In [ ]:
sat = 'thb'
coord = 'gse'
tau = timedelta(seconds=60)
data_resolution = timedelta(seconds=4)

files = f'data/{sat}_fgs_{coord}.parquet'
output = f'data/{sat}_candidates_{year}_tau_{tau.seconds}.parquet'

if os.path.exists(output):
    logger.info(f"Skipping {year} as the output file already exists.")
    continue

data = pl.scan_parquet(files).set_sorted('time').collect()
sat_fgm = df2ts(data, ["BX", "BY", "BZ"], attrs={"coordinate_system": coord, "units": "nT"})
sat_state = pandas.read_parquet(f'data/{sat}_state.parquet')

indices = compute_indices(data, tau)
# filter condition
sparse_num = tau / data_resolution // 3
filter_condition = get_ID_filter_condition(sparse_num = sparse_num)

candidates = indices.filter(filter_condition).with_columns(pl_format_time(tau))

ids = process_candidates(candidates, sat_fgm, sat_state)
df = ids.to_pandas()
df.to_parquet(output)

  0%|          | 0/6 [00:00<?, ?it/s]21-Sep-23 03:05:26: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.

Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
  0%|          | 0/6 [01:39<?, ?it/s]


KeyboardInterrupt: 

## Results

In [ ]:
# read candidates from files in current directory
pattern = 'data/candidates*.parquet'
data = Path() / pattern

candidates = pl.scan_parquet(data).collect()

In [ ]:
len(candidates.columns)

### Plotting candidates of different types of discontinuities

In [ ]:
plot_candidates(candidates, candidate_type='TD')

In [ ]:
plot_candidates(candidates, candidate_type='RD')

In [ ]:
plot_candidates(candidates, candidate_type='ED')

In [ ]:
plot_candidates(candidates, candidate_type='ND')

### Occurrence rates

In [ ]:
# calculate the occurence rates of different types of ID
def occurence_rate(candidates, candidate_type):
    return len(candidates[candidates['type'] == candidate_type]) / len(candidates)

def time_occurence_rate(candidates):
    if len(candidates) <= 1:
        return None
    else:
        return (candidates.iloc[-1]['tstop'] - candidates.iloc[0]['tstart']) / (len(candidates) -1)

CANDIDATE_TYPES = ['RD', 'TD', 'ED', 'ND']

for candidate_type in CANDIDATE_TYPES:
    logger.info(f"Occurrence rate of {candidate_type}: {occurence_rate(candidates, candidate_type)}")
    logger.info(f"Time occurrence rate of {candidate_type}: {time_occurence_rate(candidates[candidates['type'] == candidate_type])}")

In [ ]:
pdp.ColByFrameFunc("R", lambda df: df[['X','Y', 'Z']].apply(np.linalg.norm, axis=1), func_desc='calculating R')(candidates)

In [ ]:
candidates.plot(x="X", y="d_star")

### Duration

In [ ]:
# candidates.update(pdp_calibrate_duration.apply(temp_candidates))

### Waiting time

### Amplitude

In [ ]:
temp_candidates = get_candidates(candidates, 'RD')
temp_candidates = pdp_calc_duration(temp_candidates)
temp_candidates

In [ ]:
plot_candidates(temp_candidates)

In [ ]:
# test minvar and principal axes vectors
test_data = np.array([[1,1,0],[-1,-1,0]])
vrot, v, w = minvar(test_data)
Vi = v[:,0]
print(Vi)

In [ ]:
# test minvar_matrix_make
in_var_name = "fgm"

vrot, v, w = minvar(get_data(in_var_name, xarray=True))

minvar_matrix_make(in_var_name)
tvector_rotate(f'{in_var_name}_mva_mat', in_var_name)
(get_data(f"{in_var_name}_rot").y==vrot).all()

### Cases

In [ ]:
# plot_candidates(candidates.loc[lambda _: _['time']=='2012-07-10 02:31:15'])
temp_trange = ['2012-07-15 03:44', '2012-07-15 03:47']
temp_data = sat_fgm.sel(time=slice(*temp_trange))
temp_data.plot.scatter(x='time', hue='v_dim')
# temp_data.resample(time=pd.Timedelta(tau, unit='s')).map(calc_vec_std)
compute_index_std(temp_data, tau)

#### Case: neighboring data is missing, causing the calculation of the standard deviation index to be Inf

In [ ]:
# Case: neighboring data is missing, causing the calculation of the standard deviation index to be Inf.
temp_trange = ['2012-07-10 02:30', '2012-07-10 02:32']
temp_data = sat_fgm.sel(time=slice(*temp_trange))
temp_data.plot.scatter(x='time', hue='v_dim')
# temp_data.resample(time=pd.Timedelta(tau, unit='s')).map(calc_vec_std)
compute_index_std(temp_data, tau)

#### Caveats

In [ ]:
plot_candidates(candidates)

##### NOTE: Not very accurate for waving magnetic field...

In [ ]:
temp_candidate = {'time': Timestamp('2012-05-01 00:39:12'),
 'tstart': Timestamp('2012-05-01 00:38:56'),
 'tstop': Timestamp('2012-05-01 00:39:28'),
 'i1': 2.891042053414383,
 'i2': 2.389699609352786,
 'i3': 1.3916002784658887,
 'd_star': 0.27143595,
 'd_time': Timestamp('2012-05-01 00:39:18.672000'),
 'd_tstart': Timestamp('2012-05-01 00:39:14.672000'),
 'd_tstop': Timestamp('2012-05-01 00:39:19.671000'),
}

plot_candidate(temp_candidate)

In [ ]:
data = get_candidate_data_xr(temp_candidate, neighbor=1)
vec_diff = data.differentiate("time", datetime_unit="s", edge_order=2).isel(time=slice(1,-1))
vec_diff_mag = linalg.norm(vec_diff, dims='v_dim')
vec_diff_mag.plot()

##### NOTE: Small threshold_ratio values will tend to make the duration longer if the duration can be determined.


In [ ]:
# Test different threshold ratios
threshold_ratios = [1/8, 1/4, 0.3, 1/3, 1/2]
for threshold_ratio in threshold_ratios:
    temp_candidate.update(calc_duration(get_candidate_data_xr(temp_candidate), threshold_ratio=threshold_ratio).to_dict())
    plot_candidate(temp_candidate)